# Initialize   

In [1]:
import numpy as np
import random
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import r2_score

# SEIR Class

In [2]:
class SEIRModel:
    def __init__(self, population, beta, sigma, gamma):
        self.population = population
        self.beta = beta
        self.sigma = sigma
        self.gamma = gamma
        self.s = 100 
        self.e = 10
        self.i = 0
        self.r = 0

    def step(self):
        new_infections = self.beta * self.s * self.i / self.population
        new_exposed = new_infections
        new_recovered = self.gamma * self.i
        self.s -= new_infections
        self.e += new_exposed
        self.i += new_infections - new_recovered
        self.r += new_recovered

    def get_infected(self):
        return int(self.i)

    def __call__(self, y, t, N, beta, sigma, gamma):
        S, E, I, R = y
        dSdt = -beta * S * I / N
        dEdt = beta * S * I / N - sigma * E
        dIdt = sigma * E - gamma * I
        dRdt = gamma * I
        return dSdt, dEdt, dIdt, dRdt

# Load data

In [3]:
def load_df(data_name: str) -> pd.DataFrame:
  path = f"nb_datasets/{data_name}"

  df:pd.DataFrame = pd.read_csv(path)

  # ensure sort by date
  df['date'] = pd.to_datetime(df['date'], dayfirst=True, format='%d/%m/%Y')
  df_sorted: pd.DataFrame = df.sort_values(by='date')
  
  return df_sorted

# Split data

In [4]:
def load_observed(df: pd.DataFrame, brgy_name: str) -> pd.DataFrame:
  
  df_sorted: pd.DataFrame = df[df['barangay_Res'] == brgy_name]

  # keep relevant cols
  keep_cols = ["date", "count_Exposed", "count_Infectious", "count_Recovered"]
  main_df: pd.DataFrame = df_sorted[df_sorted.columns.intersection(keep_cols)]

  # Observed data (e.g., daily infections)
  # observed_data = main_df['count_Infectious'].to_numpy()

  # total_cases = main_df['count_Infectious'].sum()
  
  return main_df


# Sim Anneal Code

In [5]:
# Simulate SEIR model
def simulate_seir(beta, sigma, gamma, y0, t, compartment):
    N = 32841 # From searching (how to get from data?)
    seir_model = SEIRModel(N, beta, sigma, gamma)
    
    # Solve ODEs for SEIR
    result = odeint(seir_model, y0, t, args=(N, beta, sigma, gamma))
    S, E, I, R = result.T
    
    if compartment == 'E':
        return E
    elif compartment == 'I':
        return I
    elif compartment == 'R':
        return R
    else:
        print("HUH?")
        return I
      

# Objective function
def objective_function(params, observed_data, initial_conditions, time_points, compartment):
    beta, sigma, gamma = params

    # Simulate the SEIR model with given parameters
    predicted_data = simulate_seir(beta, sigma, gamma, initial_conditions, time_points, compartment)
    #print(f"pred: {len(predicted_data)} obs:{len(observed_data)}")
    r2 = r2_score(predicted_data, observed_data)
    return r2


# Simulated Annealing for SEIR parameter tuning
def simulated_annealing_seir(observed_data, initial_conditions, time_points, temp, cooling_rate, max_iter, compartment):
    # Initial parameter guesses
    current_params = [0.3, 0.1, 0.1]  # Initial guesses for beta, sigma, gamma
    current_cost = objective_function(current_params, observed_data, initial_conditions, time_points, compartment)
    best_params = current_params
    best_cost = current_cost

    for i in range(max_iter):
        # Generate new candidate parameters
        new_params = [param + random.uniform(-0.005, 0.005) for param in current_params]
        new_params = np.clip(new_params, 0, 1)  # Keep parameters in a valid range (0 to 1)

        # Evaluate the new candidate solution
        new_cost = objective_function(new_params, observed_data, initial_conditions, time_points, compartment)

        # Decide whether to accept the new solution
        if new_cost < current_cost or random.random() < np.exp((current_cost - new_cost) / temp):
            current_params = new_params
            current_cost = new_cost

            # Update the best solution found so far
            if new_cost < best_cost:
                best_params = new_params
                best_cost = new_cost

        # Cool down the temperature
        temp *= cooling_rate

    return best_params, best_cost

# Run Sim Anneal

In [6]:
brgy_name = "LANDAYAN"
main_df: pd.DataFrame = load_df("CITY_OF_SAN_PEDRO_processed.csv")
main_observed: pd.DataFrame = load_observed(main_df, brgy_name) # Change to check for other Brgys.

train_data = main_observed[(main_observed['date'] >= '01/01/2021') & (main_observed['date'] <= '10/01/2021')]
test_data = main_observed[(main_observed['date'] > '10/02/2021') & (main_observed['date'] <= '12/31/2021')]

df_observed_data = main_observed[(main_observed['date'] >= '01/01/2021') & (main_observed['date'] <= '12/31/2021')]

# Initial parameters
N = 33235  # Population from google search (how to get from data?)
I0 = train_data['count_Infectious'].iloc[0] # Initial infected from data
E0 = train_data['count_Exposed'].iloc[0] # Initial exposed cases
R0 = train_data['count_Recovered'].iloc[0] # Initial recovered
S0 = N - I0 - E0 - R0  # Susceptible

initial_conditions: tuple = [S0, E0, I0, R0]

# Time points
time_points = np.arange(train_data.shape[0]) # odeint expects an array (vector) of integers.

# Simulated Annealing parameters
temp = 10000           # Initial temperature
cooling_rate = 0.90   # Cooling rate
max_iter = 10000       # Number of iterations

parms_list = []
cost_list = []

observed_dict = {
    "E": train_data['count_Exposed'].to_numpy(),
    "I": train_data['count_Infectious'].to_numpy(),
    "R": train_data['count_Recovered'].to_numpy()
}

for compartment, observed_data in observed_dict.items():
    # Run the Simulated Annealing algorithm
    best_params, best_cost = simulated_annealing_seir(observed_data, initial_conditions, time_points, temp, cooling_rate, max_iter, compartment)
    parms_list.append(best_params)
    cost_list.append(best_cost)
    o_beta, o_sigma, o_gamma = best_params

    # Display results
    print(f"Compartment: {compartment}")
    print(f"Best parameters: Beta={o_beta:.3f}, Sigma={o_sigma:.3f}, Gamma={o_gamma:.3f}")
    print(f"Best cost (r-squared): {best_cost:.3f}")

# o_beta, o_sigma, o_gamma = 0.951, 1.000, 0.168 # from best params
# time_points = np.arange(df_observed_data.shape[0]) # odeint expects an array (vector) of integers.

# plot_actual = pd.DataFrame({
#     'date_train': train_data['date'],
#     'date_test': test_data['date'],
#     'train_exposed': train_data['count_Exposed'],
#     'test_exposed': test_data['count_Exposed'],
#     'train_infectious': train_data['count_Infectious'],
#     'test_infectious': test_data['count_Infectious'],
#     'train_recovered': train_data['count_Recovered'],
#     'test_recovered': test_data['count_Recovered']
# })

# plot_prediction = pd.DataFrame({
#     'date': df_observed_data['date']
# })

# # Visualize the results
# for compartment in observed_dict.keys():
#     predicted_data = simulate_seir(o_beta, o_sigma, o_gamma, initial_conditions, time_points, compartment)
#     plot_prediction[compartment] = predicted_data    
    

# plt.figure(figsize=(12, 6))
# plt.plot(plot_actual['date_train'], plot_actual['train_exposed'], 'r--', label='Train Exposed')
# plt.plot(plot_actual['date_test'], plot_actual['test_exposed'], 'r-', label='Test Exposed')
# plt.plot(plot_prediction['date'], plot_prediction['E'], 'ro', label='Predicted Exposed')
# plt.plot(plot_actual['date_train'], plot_actual['train_infectious'], 'b--', label='Train Infectious')
# plt.plot(plot_actual['date_test'], plot_actual['test_infectious'], 'b-', label='Test Infectious')
# plt.plot(plot_prediction['date'], plot_prediction['I'], 'bo', label='Predicted Infected')
# plt.plot(plot_actual['date_train'], plot_actual['train_recovered'], 'g--', label='Train Recovered')
# plt.plot(plot_actual['date_test'], plot_actual['test_recovered'], 'g-', label='Test Recovered')
# plt.plot(plot_prediction['date'], plot_prediction['R'], 'go', label='Predicted Recovered')
# plt.xlabel('Days')
# plt.ylabel('Population')
# plt.legend()
# plt.xticks(rotation=45)
# plt.title(f'SEIR Model Fitting For {brgy_name}')
# plt.show()

Compartment: E
Best parameters: Beta=1.000, Sigma=0.151, Gamma=0.790
Best cost (r-squared): -1.192
Compartment: I
Best parameters: Beta=1.000, Sigma=1.000, Gamma=0.000
Best cost (r-squared): -19.345
Compartment: R
Best parameters: Beta=1.000, Sigma=1.000, Gamma=0.168
Best cost (r-squared): -12.284
